<img width="8%" alt="Operations" src="https://naasai-public.s3.eu-west-3.amazonaws.com/abi-demo/operations_efficiency.png" style="border-radius: 15%">

# Pipeline

**Tags:** #naas #pipeline #jupyter #notebook #dataanalysis #workflow #streamline

**Author:** [Florent Ravenel](https://www.linkedin.com/in/florent-ravenel)

**Description:** This notebook creates a notebooks pipeline.

## Input

### Import libraries

In [ ]:
from datetime import date
import os
import glob
import naas_data_product
import naas_python

### Setup variables

In [ ]:
# Storage
datalake_dir = naas_python.secret.get("ABI_DATALAKE_DIR").value
api_key = naas_python.secret.get('NAAS_API_TOKEN').value

# Entity
entity_index = "0"
entity_name = pload(os.path.join(datalake_dir, "entities", entity_index), "entity_name") or ""
emails = pload(os.path.join(datalake_dir, "entities", entity_index), "emails") or []
entity_dir = pload(os.path.join(datalake_dir, "entities", entity_index), "entity_dir") or ""

# Google Sheets
spreadsheet_url = pload(os.path.join(datalake_dir, "entities", entity_index), "abi_spreadsheet") or ""
sheet_content = "OPS"

# Engine
engine_name = "operations-engine"
assistant_name = "Operations Assistant"
custom_pipeline_path = os.path.join(naas_data_product.MODELS_PATH, engine_name, "custom", "__pipeline__.ipynb")

## Model

### Create output directory by date

In [ ]:
engine_dir = os.path.join(entity_dir, engine_name, date.today().isoformat())
print("✅ Output directory:", engine_dir)
pipeline_dir = os.path.join(engine_dir, "pipeline_executions")
print("✅ Pipeline directory:", pipeline_dir)

### Setup notebooks
This section is made to declare all the notebooks and create unique IDs that will be used in the pipeline. 

In [ ]:
plugin = NotebookStep(
    name="Plugin",
    notebook_path=os.path.join(naas_data_product.MODELS_PATH, engine_name, "__plugin__.ipynb"),
    parameters={
        "input_dir": engine_dir,
        "name": assistant_name,
        "entity_name": entity_name,
        "output_dir": os.path.join(entity_dir, "plugins")
    }
)

if os.path.exists(custom_pipeline_path):
    custom = NotebookStep(
        name="Custom",
        notebook_path=os.path.join(naas_data_product.MODELS_PATH, engine_name, "custom", "__pipeline__.ipynb"),
        parameters={
            "entity_name": entity_name,
            "entity_dir": entity_dir,
        }
    )
else:
    custom = DummyStep("Custom")

## Output

### Run pipeline

In [ ]:
pipeline = Pipeline()

pipeline >> plugin >> custom >> End()

pipeline.run(outputs_path=pipeline_dir)